# Big data file formats

Version: 27 Sep 2022

Sandbox to play around and convert data to various formats:
* Avro
* ORC
* Parquet
* Protobuf

This Python environment was set up using conda with the following commands:
```
conda create -n formats
conda activate formats
conda install -c conda-forge pyarrow
#conda install -c conda-forge python-avro
conda install -c conda-forge pandavro
conda install -c anaconda protobuf
conda install -c anaconda jupyter
conda install -c anaconda pandas
```

## Parquet

Uses Apache Arrow

https://arrow.apache.org/docs/python/parquet.html

In [14]:
import pandas as pd

from pyarrow import json as pj
import pyarrow as pa
import pyarrow.parquet as pq

In [6]:
fn = 'kttm-2019-08-22-small-mod.json.gz'
table = pj.read_json(fn)
table
table.to_pandas()

,timestamp,city,country,session,session_length
0,2019-08-22T00:00:00.212Z,Christchurch,New Zealand,S97953932,337
1,2019-08-22T00:00:00.327Z,Portland,United States,S59897315,37269
2,2019-08-22T00:00:00.387Z,Istanbul,Turkey,S16418954,20591
3,2019-08-22T00:00:00.740Z,Brasília,Brazil,S32580935,18700
4,2019-08-22T00:00:01.035Z,Arichuna,Venezuela,S57539178,20125
...,...,...,...,...,...
274855,2019-08-22T23:59:57.573Z,Sugar Land,United States,S29804301,71153
274856,2019-08-22T23:59:57.886Z,Washington,United States,S22531821,10668
274857,2019-08-22T23:59:58.347Z,,United States,S25042020,65133
274858,2019-08-22T23:59:59.120Z,Banora Point,Australia,S45081833,258788


In [3]:
pq.write_table(table, 'kttm-2019-08-22-small-mod.parquet')

## ORC

Uses Apache Arrow

https://arrow.apache.org/docs/python/orc.html

In [2]:
from pyarrow import orc

In [7]:
orc.write_table(table, 'kttm-2019-08-22-small-mod.orc')

## Avro

https://www.perfectlyrandom.org/2019/11/29/handling-avro-files-in-python/
Seems complicated.

https://github.com/ynqa/pandavro

In [1]:
import avro

In [8]:
import pandavro as pdx

In [12]:
df = table.to_pandas()

In [13]:
OUTPUT_PATH = 'kttm-2019-08-22-small-mod.avro'
pdx.to_avro(OUTPUT_PATH, df)
saved = pdx.read_avro(OUTPUT_PATH)
print(saved)

                       timestamp          city        country    session  \
0       2019-08-22T00:00:00.212Z  Christchurch    New Zealand  S97953932   
1       2019-08-22T00:00:00.327Z      Portland  United States  S59897315   
2       2019-08-22T00:00:00.387Z      Istanbul         Turkey  S16418954   
3       2019-08-22T00:00:00.740Z      Brasília         Brazil  S32580935   
4       2019-08-22T00:00:01.035Z      Arichuna      Venezuela  S57539178   
...                          ...           ...            ...        ...   
274855  2019-08-22T23:59:57.573Z    Sugar Land  United States  S29804301   
274856  2019-08-22T23:59:57.886Z    Washington  United States  S22531821   
274857  2019-08-22T23:59:58.347Z                United States  S25042020   
274858  2019-08-22T23:59:59.120Z  Banora Point      Australia  S45081833   
274859  2019-08-22T23:59:59.949Z                United States  S25042020   

        session_length  
0                  337  
1                37269  
2           

## Protobuf

https://developers.google.com/protocol-buffers/docs/overview#work

https://druid.apache.org/docs/latest/development/extensions-core/protobuf.html

In [15]:
import json

from google.protobuf.json_format import Parse, ParseDict

In [19]:
import gzip

In [34]:
data = []
with gzip.open(fn, 'r') as fin:
    #data = json.loads(fin.read().decode('utf-8'))
    
    contents = fin.read().decode('utf-8')

    #data = [json.loads(str(item)) for item in contents.strip().split('\n')]
    
    for line in contents.strip().split('\n'):
        try:
            data.append(json.loads(line))
        except:
            continue

# https://stackoverflow.com/questions/39450065/python-3-read-write-compressed-json-objects-from-to-gzip-file
# https://stackoverflow.com/questions/21533894/how-to-read-line-delimited-json-from-large-file-line-by-line
# https://stackoverflow.com/questions/51300674/converting-json-into-newline-delimited-json-in-python

In [35]:
data

[{'timestamp': '2019-08-22T00:00:00.212Z',
  'city': 'Christchurch',
  'country': 'New Zealand',
  'session': 'S97953932',
  'session_length': 337},
 {'timestamp': '2019-08-22T00:00:00.327Z',
  'city': 'Portland',
  'country': 'United States',
  'session': 'S59897315',
  'session_length': 37269},
 {'timestamp': '2019-08-22T00:00:00.387Z',
  'city': 'Istanbul',
  'country': 'Turkey',
  'session': 'S16418954',
  'session_length': 20591},
 {'timestamp': '2019-08-22T00:00:00.740Z',
  'city': 'Brasília',
  'country': 'Brazil',
  'session': 'S32580935',
  'session_length': 18700},
 {'timestamp': '2019-08-22T00:00:01.035Z',
  'city': 'Arichuna',
  'country': 'Venezuela',
  'session': 'S57539178',
  'session_length': 20125},
 {'timestamp': '2019-08-22T00:00:01.486Z',
  'city': 'San Jose',
  'country': 'United States',
  'session': 'S18064314',
  'session_length': 10385},
 {'timestamp': '2019-08-22T00:00:01.512Z',
  'city': 'Oklahoma City',
  'country': 'United States',
  'session': 'S60897860'

In [36]:
# protobuf schema
message KTTM {
    string timestamp = 1;
    string city = 2;
    string country = 3;
    int32 session = 4;
    int32 session_length = 5;
}

SyntaxError: invalid syntax (1010908654.py, line 2)

In [38]:
d = data[0]
d

{'timestamp': '2019-08-22T00:00:00.212Z',
 'city': 'Christchurch',
 'country': 'New Zealand',
 'session': 'S97953932',
 'session_length': 337}

In [39]:
message = ParseDict(d, Thing())
#message = Parse(json.dumps(d), Thing())    

print(message.first)  # "a string"
print(message.second) # True
print(message.third)  # 123456789

NameError: name 'Thing' is not defined

In [ ]:
ess